In [66]:
#First, load in the necessary libraries - we're going to use Pandas to explore and preprocess the dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
prop = fm.FontProperties(fname='../extras/fonts/Nunito-Medium.ttf')
soft_white = '#FBFAF5'
discord_dark = '#424549'

#Get our machine learning libraries
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
pd.set_option('display.max_columns', None)

In [55]:
#Next, load in the dataset from the url below. After doing so, print out each column and it's related data type (dtype) as well as a small data sample
#The dataset is a breakdown of charges to an individual's insurance, and includes characteristics of the individual that are age, sex, bmi, # of children, smoker, and region.
df = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
display(df.head())
print('-----------------')
for c in df:
    print(c, df[c].dtype)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


-----------------
age int64
sex object
bmi float64
children int64
smoker object
region object
charges float64


In [97]:
#Preprocess data - #Start by converting string to int
df['sex'] = pd.Categorical(df['sex']).codes
df['children'] = pd.Categorical(df['children']).codes
df['smoker'] = pd.Categorical(df['smoker']).codes
df['region'] = pd.Categorical(df['region']).codes

#Create transformer
transformer = make_column_transformer(
    (MinMaxScaler(), ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'])
)

#Transform and split data
transformer.fit(df)
data = transformer.transform(df)

#Split data
y_data = data[:,-1]
X_data = df.iloc[:,:-1]

test_size = 100
X_train, X_test = X_data[:-test_size], X_data[-test_size:]
y_train, y_test = y_data[:-test_size], y_data[-test_size:]

#What does our data look like now?
display(df.head())

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [99]:
#Create model
model = Sequential()
model.add(Dense(256, input_shape=(X_data.shape[1],), activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(1))

#Compile and fit model
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics='mse')
model.fit(X_train, y_train, batch_size=8, epochs=50)

Epoch 1/50
155/155 [==============================] - 1s 5ms/step - loss: 0.3597 - mse: 0.3597
Epoch 2/50
155/155 [==============================] - 0s 3ms/step - loss: 0.0192 - mse: 0.0192
Epoch 3/50
155/155 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0.0202
Epoch 4/50
155/155 [==============================] - 0s 3ms/step - loss: 0.0449 - mse: 0.0449
Epoch 5/50
155/155 [==============================] - 0s 3ms/step - loss: 0.0409 - mse: 0.0409
Epoch 6/50
155/155 [==============================] - 0s 3ms/step - loss: 0.0409 - mse: 0.0409
Epoch 7/50
155/155 [==============================] - 0s 3ms/step - loss: 0.0358 - mse: 0.0358
Epoch 8/50
155/155 [==============================] - 1s 3ms/step - loss: 0.0356 - mse: 0.0356
Epoch 9/50
155/155 [==============================] - 1s 3ms/step - loss: 0.0271 - mse: 0.0271
Epoch 10/50
155/155 [==============================] - 1s 4ms/step - loss: 0.0300 - mse: 0.0300
Epoch 11/50
155/155 [============================

In [85]:
#Get predictions and mse
preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)
x = np.arange(len(preds))

#Plot data
plt.figure(facecolor=soft_white)
ax = plt.axes()
ax.plot(x, y_test, label='True')
ax.plot(x, preds, label='Preds')

#Styling
ax.set_facecolor(soft_white)
ax.set_xlabel('Sample Number', fontproperties=prop)
ax.set_ylabel('Mean Squared Error (MSE)', fontproperties=prop)
ax.set_title(f'Regression Neural Network MSE ({round(mse, 5)})')
for label in ax.get_xticklabels():
    label.set_fontproperties(prop)
for label in ax.get_yticklabels():
    label.set_fontproperties(prop)

ax.legend()
plt.savefig('../extras/figures/nn_mse.png')

4/4 [==============================] - 0s 3ms/step


In [102]:
#sklearn regression - SVM, SGD, KNN, Gaussian, Bayesian, Decision Tree, AdaBoostRegressor, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor

ensembles = [('adaboost_mse.png', AdaBoostRegressor()), ('randomforest_mse.png', RandomForestRegressor()), ('gradientboost_mse.png', GradientBoostingRegressor())]
for savepath, e in ensembles:
    e.fit(X_train, y_train)
    preds = e.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    print(e, mse)

    x = np.arange(len(preds))
    
    #Plot data
    plt.figure(facecolor=soft_white)
    ax = plt.axes()
    ax.plot(x, y_test, label='True')
    ax.plot(x, preds, label='Preds')

    #Styling
    ax.set_facecolor(soft_white)
    ax.set_xlabel('Sample Number', fontproperties=prop)
    ax.set_ylabel('Mean Squared Error (MSE)', fontproperties=prop)
    ax.set_title(f'{str(e)[:-2]} MSE ({round(mse, 5)})')
    for label in ax.get_xticklabels():
        label.set_fontproperties(prop)
    for label in ax.get_yticklabels():
        label.set_fontproperties(prop)

    ax.legend()
    plt.savefig(f'../extras/figures/{savepath}')


BayesianRidge() 0.010070940238109234
AdaBoostRegressor() 0.006146852104194975
RandomForestRegressor() 0.003976904703289654
GradientBoostingRegressor() 0.00463148916592886
